In [1]:
import os
import datetime

import numpy as np
import pandas as pd
import xarray as xr

import datacube
dc = datacube.Datacube()


In [2]:
fire_events = pd.read_csv('fire_events.csv', index_col=0)
fire_events

,lon_min,lon_max,lat_min,lat_max,time_min,time_max,fire_date,sensor
EventName,,,,,,,,
BlackSaturday,145.0000,145.9000,-37.2000,-37.7000,2008-01,2009-12,2009-02-07,ls5
Tas2016_Central_Plateau,146.0852,146.3702,-41.5985,-41.7314,2013-01,2018-01,2016-02-08,ls8
Tas2016_Sumac_Forest,144.6893,145.1582,-41.0711,-41.3711,2013-01,2018-01,2016-02-08,ls8


In [3]:
   
def _get_Pixel_Quality():
    """This calls the dc.load function on the pixel quality product.
    This uses the sensor specified in the 'fire_events.csv' file."""

    start_time = datetime.datetime.now()
    pq = dc.load( product=  sensor + '_pq_albers', 
           x=lon_extent, 
           y=lat_extent,
           time = time_bounds,
           resolution = (-100,100)
      #      dask_chunks = {'time': 5}
                     )
    end_time = datetime.datetime.now()

    # This next chunk creates a cloud_mask using the datacube masking methods
    from datacube.storage import masking
    cloud_mask = masking.make_mask(pq, 
        cloud_acca='no_cloud', cloud_fmask='no_cloud',
        cloud_shadow_acca = 'no_cloud_shadow', cloud_shadow_fmask = 'no_cloud_shadow',
        # contiguous=True ,land_sea = 'land'
        ).pixelquality
    
    print ("cloud mask loading took: ", (end_time - start_time))
    print ("cloud mask: ", (cloud_mask).nbytes / 10**9 , "GB")
      
    
    return cloud_mask



def get_Reflectance():
    """This calls the dc.load function on the NBAR reflectance product.
    This uses the sensor specified in the 'fire_events.csv' file."""

    start_time = datetime.datetime.now()

    reflectance = dc.load( product=  sensor + '_nbar_albers', 
               x=lon_extent, 
               y=lat_extent,
               time = time_bounds,
               resolution = (-100,100)
      #          dask_chunks = {'time': 5}
                     )

    reflectance = reflectance.where((reflectance != -999))

    end_time = datetime.datetime.now()
    print ("reflectance loading took: ", (end_time - start_time))
    print ("reflectance: ", (reflectance).nbytes / 10**9 , "GB")
    print ('Note: access original NBAR reflectance xarray dataset by calling "(event_name).reflanctance"')
    print ('Note: access cloud-masked NBAR reflectance xarray dataset by calling "(event_name).reflanctance_cloudmasked"')
    
    reflectance.attrs['note'] = ('This is the original data from Datacube. No masking has been performed.')
    
    return (reflectance)




In [4]:
event_name = 'Tas2016_Sumac_Forest'     #ENTER HERE
name = (fire_events[fire_events.index==event_name].index.values[0] ) 
lat_extent = ( fire_events[fire_events.index==event_name].lat_min.values[0] ,
                   fire_events[fire_events.index==event_name].lat_max.values[0] )
lon_extent = ( fire_events[fire_events.index==event_name].lon_min.values[0] ,
                   fire_events[fire_events.index==event_name].lon_max.values[0] )
time_bounds = (np.datetime64('2000-01') , np.datetime64('2018-01'))
sensor = 'ls8'
#sensor = fire_events[fire_events.index==event_name].sensor.values[0]

In [5]:
A = _get_Pixel_Quality()
datacube.storage.storage.write_dataset_to_netcdf(A, ('%s_cloudmask - %s.nc') % (name, sensor))

cloud mask loading took:  0:05:21.734656
cloud mask:  0.048288576 GB


AttributeError: 'DataArray' object has no attribute 'data_vars'

In [ ]:
del(A)

sensor = 'ls5

A = _get_Pixel_Quality()
datacube.storage.storage.write_dataset_to_netcdf(A, ('%s_cloudmask - %s.nc') % (name, sensor))

In [ ]:
del(A)
time_bounds = (np.datetime64('2010-01') , np.datetime64('2014-01'))
sensor = 'ls7'

A = _get_Pixel_Quality()
datacube.storage.storage.write_dataset_to_netcdf(A, ('%s_cloudmask - %s.nc') % (name, sensor))

In [ ]:
del(A)

In [11]:
lat_extent

(-41.071100000000001, -41.371099999999998)